In [3]:
%config Completer.use_jedi=False
from importlib import reload
import pandas as pd
import numpy as np
from diff_cap_packages import Xy
from diff_cap_packages import filters
from diff_cap_packages import ta, models_
reload(Xy)

<module 'diff_cap_packages.Xy' from '/Users/varunsaini/github/Diff_Capital_Internship/diff_cap_packages/Xy.py'>

In [26]:
from sklearn.linear_model import LinearRegression
common_features = common_features = models_.get_common_features("2007-03-20", 200)
data = models_.get_dataframe_for_pycaret("930060", "2007-03-20", 200, common_features)
model = LinearRegression()
model.fit(data.iloc[1:-1, :-1].fillna(method="bfill"), data.iloc[1:-1, -1])
# data.iloc[1:-1, :-1].fillna(method="bfill").isna().any()
# data.iloc[1:-1, :-1]["36086P"]

LinearRegression()

In [ ]:
stock_id = "930060"
end_date = "2021-06-29"
days = 150

market_prices = delayed_prices = relevant_date_technicals = pd.DataFrame()

market_prices = filters.get_dataframe_prices(days, end_date, nan_threshold=0)

# delayed_prices = Xy.get_delayed_X(stock_id, end_date=end_date, days_before=days)

technicals = Xy.get_market_Xy()[0].loc[:, [stock_id]]
volume = filters.volumes.loc[:, stock_id]
ta.add_ATR(technicals, inplace=True)
ta.add_RSI(technicals, inplace=True)
# ta.add_OBV(technicals, volume, inplace=True)

technicals.drop(stock_id, axis=1, inplace=True)
relevant_date_technicals = technicals.loc[:end_date].tail(days)
relevant_date_technicals


features = pd.concat([market_prices, delayed_prices, relevant_date_technicals], axis=1)
features

In [ ]:
target = Xy.get_market_Xy(stock_id, target_ahead_by=1)[1]
target = target.loc[:end_date]
target

In [ ]:
from diff_cap_packages import models_
reload(models_)

In [ ]:
common_features = models_.get_common_features('2008-12-30', 500)
models_.get_dataframe_for_pycaret('930060', '2008-12-30', 500, common_features)

In [ ]:
from diff_cap_packages import models_
models_.get_target_for_stock('930060', '2008-12-30', 500)

In [ ]:
from pycaret.regression import *
features = models_.get_features_for_stock('930060', '2021-06-29', 200)
target = models_.get_target_for_stock('930060', '2021-06-29', 200)
data = pd.concat([features, target], axis=1)
setup(data=data, target=data.columns[-1], train_size=0.8)

In [ ]:
best = compare_models()

In [ ]:
prep_pipe = get_config('prep_pipe')
transformed = prep_pipe.transform(features)

# ax = target.plot()
# pd.DataFrame(best.predict(transformed), index=future_idx).plot(ax=ax)

# pd.DataFrame(best.predict(transformed), index=future_idx, columns=['tomorrow'])

predicted = pd.DataFrame(best.predict(transformed), index=target.index)
results = pd.concat([features.loc[:, '930060'], target, predicted], axis=1)
results.columns = ['930060', 'actual next day', 'predicted next day']
results["actual % chg"] = (results["actual next day"] - results["930060"]) / results["930060"]
results["pred % chg"] = (results["predicted next day"] - results["930060"]) / results["930060"]
results["% error"] = (results["pred % chg"] - results["actual % chg"]) / results["actual % chg"]
results["buy?"] = results["actual % chg"] > 0
results["pred buy?"] = results["pred % chg"] > 0
# results.replace([np.inf, -np.inf], 0).mean()
results['accurate'] = results["buy?"] == results["pred buy?"]
results.sum()

# TESTING MODELS_.PY

In [ ]:
from diff_cap_packages import models_

In [ ]:
data = models_.get_dataframe_for_pycaret("930060", "2021-06-29", 500)

In [ ]:
models_.setup_pycaret(data)

In [ ]:
best = models_.get_best_model(data)

In [ ]:
from diff_cap_packages import models_
from pycaret.regression import *
best = load_model("models/test_save")

features = models_.get_features_for_stock('930060', '2021-06-29', 200)
target = models_.get_target_for_stock('930060', '2021-06-29', 200)

results = models_.get_analysis(best, '930060', features, target)
models_.get_accurate_predictions(results)

# SCALING

In [ ]:
from diff_cap_packages import models_
from pycaret.regression import *

end_date = '2021-06-29'
days = 500

investible_universe = models_.get_investible_universe(end_date, days)

In [ ]:
investible_universe

In [ ]:
# THIS TAKES A LOT OF TIME (MORE THAN ONE HOUR)
# Make sure that, if you run this, you do it with a GPU (in Google Colab, for example)

for count, stock in enumerate(investible_universe):
  print(str(count + 1), "out of", len(investible_universe))
  data = models_.get_dataframe_for_pycaret(stock, "2021-06-29", 500)
  models_.setup_pycaret(data)
#   best = models_.get_best_model(data)
  best = create_model("lr")
  path_to_save = "models/2021-06-29-linear-500test/" + stock
  save_model(best, path_to_save)
  print("SAVED", stock)

# Ranking

## Using the Linear Regression Models

In [ ]:
from diff_cap_packages import models_
from pycaret.regression import *
import pandas as pd

end_date = '2021-06-29'
days = 500

investible_universe = models_.get_investible_universe(end_date, days)
lrmodels = pd.DataFrame(index=investible_universe)

for stock in investible_universe:
    lrmodels.loc[stock, "linear regression model"] = load_model("models/2021-06-29-linear-500/" + stock)

In [ ]:
for stock in investible_universe:
    model = lrmodels.loc[stock, "linear regression model"]
    features = models_.get_features_for_stock(stock, end_date, days)
    target = models_.get_target_for_stock(stock, end_date, days)
    
    prediction = predict_model(model, features)
    from sklearn.metrics import r2_score
    r2 = r2_score(target, prediction["Label"])
    lrmodels.loc[stock, "r2"] = r2
    
    results = models_.get_analysis(model, stock, features, target)
    accurate_predictions = models_.get_accurate_predictions(results)
    lrmodels.loc[stock, "prop_buy/sell_correct"] = accurate_predictions[0]
    
    print(stock, r2, accurate_predictions[0])

In [ ]:
models_sorted = lrmodels.sort_values("prop_buy/sell_correct", ascending=False)
models_sorted

In [ ]:
predictions = pd.DataFrame(index=lrmodels.index, columns=["predicted % chg", "adjusted predicted % chg"])

for stock in investible_universe:
    date = "2021-06-29" # Predicting for 2021-06-30
    data = models_.get_dataframe_for_pycaret(stock, end_date, days)
    model = lrmodels.loc[stock, "linear regression model"]
    pred_perc_chg = (predict_model(model, data).loc[date, "Label"] - data.loc[date, stock]) / data.loc[date, stock]
    predictions.loc[stock, "predicted % chg"] = pred_perc_chg
    predictions.loc[stock, "adjusted predicted % chg"] = pred_perc_chg * lrmodels.loc[stock, "prop_buy/sell_correct"]
    predictions.loc[stock, "actual % chg"] = (data.loc[date, stock + " +1 day"] - data.loc[date, stock]) / data.loc[date, stock]
    

In [ ]:
predictions_sorted = predictions.sort_values("adjusted predicted % chg")
predictions_sorted = predictions_sorted[predictions_sorted["predicted % chg"].abs() < 0.08]

buy = predictions_sorted.tail(5)
buy["predicted action"] = "BUY"
sell = predictions_sorted.head(5)
sell["predicted action"] = "SELL"
buy_sell = pd.concat([buy, sell])

predictions_sorted

In [ ]:
for stock in buy_sell.index:
    model = lrmodels.loc[stock, "linear regression model"]
    features = models_.get_features_for_stock(stock, end_date, days)
    target = models_.get_target_for_stock(stock, end_date, days)

    results = models_.get_analysis(model, stock, features, target)
    if results.loc[end_date, "buy?"] == True:
        buy_sell.loc[stock, "proper action"] = "BUY"
    else:
        buy_sell.loc[stock, "proper action"] = "SELL"
    
    buy_sell["correct?"] = buy_sell["predicted action"] == buy_sell["proper action"]
       
    
print("Cumulative Profit %:", buy_sell.head(5)["actual % chg"].mean() - buy_sell.tail(5)["actual % chg"].mean())
buy_sell

## Using the "Best" Models

In [ ]:
from diff_cap_packages import models_
from pycaret.regression import *

end_date = '2021-06-29'
days = 200

investible_universe = models_.get_investible_universe(end_date, days)
best_models = pd.DataFrame(index=investible_universe)

for stock in investible_universe:
    best_models.loc[stock, "linear regression model"] = load_model("models/best_models/" + stock)

# Testing stock_selection.py

In [ ]:
from diff_cap_packages import stock_selection
import importlib
importlib.reload(stock_selection)
%config Completer.use_jedi=False

In [ ]:
# Use to load in trained models.
lrmodels = stock_selection.load_models("2021-06-29")

In [ ]:
# Gets predictions for the day AFTER the date you input. 
# This date should ideally match the date in the block above.
predictions = stock_selection.get_predictions(lrmodels, "2021-06-29")

In [ ]:
# Contains information about which stocks to buy and which to short sell for the next day.
buy_sell = stock_selection.get_buy_sell(predictions)
buy_sell

In [ ]:
# Get the stocks that the models say to buy
stock_selection.get_buys_from_buy_sell(buy_sell)

In [ ]:
# Get the stocks that the models say to short sell
stock_selection.get_sells_from_buy_sell(buy_sell)

In [ ]:
# Look at a deeper analysis of the performance of the models
# lenient_on_holds is toggleable (technically, if the stock doesn't move, it isn't a loss)
stock_selection.get_buy_sell_analysis(buy_sell, lenient_on_holds=True)

In [ ]:
# Get predicted daily % change
stock_selection.get_predicted_overall_pct_chg(buy_sell)

In [ ]:
# Get actual daily % change
stock_selection.get_actual_overall_pct_chg(buy_sell)

In [ ]:
from diff_cap_packages import models_
from sklearn.linear_model import LinearRegression
common_features = models_.get_common_features(end_date="2021-06-29", num_days=200)
data = models_.get_dataframe_for_pycaret("930060", "2021-06-29", 200, common_features)
X = data.iloc[1:-1, :-1]
y = data.iloc[1:-1, -1]
model = LinearRegression()
model.fit(X, y)

In [ ]:
Xtest = models_.get_dataframe_for_pycaret("930060", "2021-06-29", 200, common_features).iloc[[-1]].iloc[:, :-1]
model.predict(Xtest)[0]

In [ ]:
models_.get_dataframe_for_pycaret("930060", "2021-06-29", 200, common_features).iloc[[-1]].iloc[:, :-1]

In [1]:
from diff_cap_packages import stock_selection


dates = ["2010-05-10"]

num_days_back = 200
for index, end_date in enumerate(dates):
    
    print("--------------------")
    print("[", str(index + 1) + "/" + str(len(dates)), "]")
    print("Starting:", end_date + ".", "Using data from the previous", num_days_back, "days to train.")
    
    stock_selection.make_and_store_linear_models(end_date, num_days_back, force_overwrite=True, pycaret=False)

    
    lrmodels = stock_selection.load_models(end_date, num_days_back, pycaret=False)
    predictions = stock_selection.get_predictions(lrmodels, end_date, num_days_back, pycaret=False, message=False)
    
    print("Finished.")
    print("--------------------")

--------------------
[ 1/1 ]
Starting: 2010-05-10. Using data from the previous 200 days to train.
This set of models is completely trained already.
Set force_overwrite to True if you want to overwrite it.


100%|██████████| 82/82 [00:00<00:00, 5473.33it/s]

Finished.
--------------------


In [2]:
buy_sell = stock_selection.get_buy_sell(predictions)
analysis = stock_selection.get_buy_sell_analysis(buy_sell)
analysis


100%|██████████| 10/10 [00:00<00:00, 1770.27it/s]


,predicted % chg,actual % chg,predicted action,proper action,ACCURATE (HOLD = True)
317449,0.077706,0.049231,BUY,BUY,True
50762X,0.043513,-0.008060,BUY,SELL,False
317381,0.042808,0.022700,BUY,BUY,True
152204,0.037145,0.005828,BUY,BUY,True
317446,0.035047,-0.032755,BUY,SELL,False
152162,-0.036368,0.004180,SELL,BUY,False
923904,-0.036929,-0.027286,SELL,SELL,True
930523,-0.048335,-0.023102,SELL,SELL,True
950511,-0.053363,0.054054,SELL,BUY,False
916481,-0.064689,-0.023150,SELL,SELL,True
